In [ ]:
import ast

import os, sys
sys.path.insert(0, '../utils/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from multiprocessing import Pool
from functools import partial
from calculate_metrics import get_country_investments, get_country_investments_by_year, get_country_investments_by_year_multiprocess, get_countries_by_year_multiprocess
from calculate_metrics import get_total_investments_by_year_multiprocess

%matplotlib inline

In [ ]:
eq_file = '../data/equities/EQ_1998_Country.xlsx'
df = pd.read_excel(eq_file)

In [ ]:
df.head()

In [ ]:
pd.read_excel('../data/infrastructure/INF_2021_Country.xlsx')

In [ ]:
pd.read_excel('../data/real_estate/RE_2011_Country.xlsx').head(5)

In [ ]:
print(get_country_investments(input_df=df, country='Singapore', financial_sector=''))
print(get_country_investments(input_df=pd.read_excel('../data/real_estate/RE_2011_Country.xlsx'), country='France', financial_sector='real estate'))

In [ ]:
for file in os.listdir('../data/equities'):
    if 'data_here' in file: continue
    print(file)
    year = int(file.split('_')[1])
    print("Year:", year, type(year))

In [ ]:
# %timeit get_country_investments_by_year_multiprocess(data_dir='../data/equities', country='Singapore')

In [ ]:
# %timeit get_country_investments_by_year(data_dir='../data/equities', country='Singapore')

In [ ]:
# investments = get_country_investments_by_year(data_dir='../data/equities', country='Singapore')
investments = get_country_investments_by_year_multiprocess(data_dir='../data/equities', country='India')
investments = sorted(investments, key=lambda x: x[0])
investments

In [ ]:
# Extract years and values
years = [item[0] for item in investments]
values = [item[1] for item in investments]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(years, values, color='skyblue')
plt.xlabel('Year')
plt.ylabel('USD')
plt.title('India investments over the Year')
plt.grid(True, alpha=0.3)
plt.show()

# Creating Processed data

## Invested Countries by year

In [ ]:
countries_by_year = get_countries_by_year_multiprocess(data_dir='../data/equities/')
countries_by_year = sorted(countries_by_year, key=lambda x: x[0])
df_countries_equity = pd.DataFrame(countries_by_year, columns=['Year', 'Countries'])
df_countries_equity['type'] = 'equities'

countries_by_year = get_countries_by_year_multiprocess(data_dir='../data/real_estate/')
countries_by_year = sorted(countries_by_year, key=lambda x: x[0])
df_countries_real_estate = pd.DataFrame(countries_by_year, columns=['Year', 'Countries'])
df_countries_real_estate['type'] = 'real estate'

countries_by_year = get_countries_by_year_multiprocess(data_dir='../data/fixed_income/')
countries_by_year = sorted(countries_by_year, key=lambda x: x[0])
df_countries_fixed_income = pd.DataFrame(countries_by_year, columns=['Year', 'Countries'])
df_countries_fixed_income['type'] = 'fixed income'

countries_by_year = get_countries_by_year_multiprocess(data_dir='../data/infrastructure/')
countries_by_year = sorted(countries_by_year, key=lambda x: x[0])
df_countries_infrastructure = pd.DataFrame(countries_by_year, columns=['Year', 'Countries'])
df_countries_infrastructure['type'] = 'infrastructure'

pd.concat([df_countries_equity, df_countries_real_estate, df_countries_fixed_income, df_countries_infrastructure], ignore_index=False).to_csv(f'../data/processed_data/countries_by_year.csv', index=False)

## Total investments by year

In [ ]:
get_total_investments_by_year_multiprocess(data_dir='../data/real_estate/')

In [ ]:
total_by_year = get_total_investments_by_year_multiprocess(data_dir='../data/equities/')
total_by_year = sorted(total_by_year, key=lambda x: x[0])
df_equity = pd.DataFrame(total_by_year, columns=['Year', 'USD'])
df_equity['type'] = 'equities'

total_by_year = get_total_investments_by_year_multiprocess(data_dir='../data/real_estate/')
total_by_year = sorted(total_by_year, key=lambda x: x[0])
df_real_estate = pd.DataFrame(total_by_year, columns=['Year', 'USD'])
df_real_estate['type'] = 'real estate'

total_by_year = get_total_investments_by_year_multiprocess(data_dir='../data/fixed_income/')
total_by_year = sorted(total_by_year, key=lambda x: x[0])
df_fixed_income = pd.DataFrame(total_by_year, columns=['Year', 'USD'])
df_fixed_income['type'] = 'fixed income'

total_by_year = get_total_investments_by_year_multiprocess(data_dir='../data/infrastructure/')
total_by_year = sorted(total_by_year, key=lambda x: x[0])
df_infrastructure = pd.DataFrame(total_by_year, columns=['Year', 'USD'])
df_infrastructure['type'] = 'infrastructure'

pd.concat([df_equity, df_real_estate, df_fixed_income, df_infrastructure], ignore_index=False).to_csv(f'../data/processed_data/total_investments_by_year.csv', index=False)

## Country investments by year

In [ ]:
investments = get_country_investments_by_year_multiprocess(data_dir='../data/equities', country='India')
# investments = sorted(investments, key=lambda x: x[0])
# investments
eq_dict = {}
for year, investment in investments:
    eq_dict[year] = investment
eq_dict

In [ ]:
df_countries_investments = pd.DataFrame(columns=['Year', 'Country', 'equities', 'real estate', 'fixed income', 'infrastructure', 'Total Investments' ])
df_countries_investments

In [ ]:
df_countries = pd.read_csv('../data/processed_data/countries_by_year.csv')
print('Getting the Full Country list')
country_list = []
for idx, row in df_countries.iterrows():
    if len(country_list)==0:
        country_list = list(ast.literal_eval(row['Countries']))
    else:
        tmp_list = list(ast.literal_eval(row['Countries']))
        for country in tmp_list:
            if country not in country_list: country_list.append(country)
    # break
print(country_list)

In [ ]:
df_countries_investments = pd.DataFrame(columns=['Year', 'Country', 'equities', 'real estate', 'fixed income', 'infrastructure', 'Total Investments' ])
if not os.path.exists('../data/processed_data/countries_investments_by_year.csv'):
    for idx, country in enumerate(country_list):
        min_year, max_year = 3000, 0
        print(f'{idx+1}. {country=}')
        # Equities
        investments = get_country_investments_by_year_multiprocess(data_dir='../data/equities', country=country)
        eq_dict = {}
        for year, investment in investments:
            eq_dict[year] = investment
            if min_year > year: min_year = year
            if max_year < year: max_year = year
    
        # Fixed Income
        investments = get_country_investments_by_year_multiprocess(data_dir='../data/fixed_income', country=country)
        fi_dict = {}
        for year, investment in investments:
            fi_dict[year] = investment
            if min_year > year: min_year = year
            if max_year < year: max_year = year
    
        # Real Estate
        investments = get_country_investments_by_year_multiprocess(data_dir='../data/real_estate', country=country)
        re_dict = {}
        for year, investment in investments:
            re_dict[year] = investment
            if min_year > year: min_year = year
            if max_year < year: max_year = year
        
        # Infrastructure
        investments = get_country_investments_by_year_multiprocess(data_dir='../data/infrastructure', country=country)
        in_dict = {}
        for year, investment in investments:
            in_dict[year] = investment
            if min_year > year: min_year = year
            if max_year < year: max_year = year
    
        # tmp_dict = {'Year', 'Country', 'equities', 'real estate', 'fixed income', 'infrastructure', 'Total Investments'}
        df_country_investments = pd.DataFrame(columns=['Year', 'Country', 'equities', 'real estate', 'fixed income', 'infrastructure', 'Total Investments' ])
        for year in range(min_year, max_year+1):
            tmp_dict = {'Year': year, 'Country': country,
                        'equities': eq_dict[year] if year in eq_dict else 0,
                        'real estate': re_dict[year] if year in re_dict else 0,
                        'fixed income': fi_dict[year] if year in fi_dict else 0,
                        'infrastructure': in_dict[year] if year in in_dict else 0,
                       }
            tmp_dict['Total Investments'] = tmp_dict['equities'] + tmp_dict['real estate'] + tmp_dict['fixed income'] + tmp_dict['infrastructure']
            tmp_df = pd.DataFrame([tmp_dict])
            df_country_investments = pd.concat([df_country_investments, tmp_df], ignore_index=True)
        # print(min_year, max_year)
        df_countries_investments = pd.concat([df_countries_investments, df_country_investments], ignore_index=True)
        # if idx > 1 : break
    df_countries_investments
    df_countries_investments.to_csv('../data/processed_data/countries_investments_by_year.csv')

In [ ]:
os.path.exists('../data/processed_data/countries_investments_by_year.csv')

### Veirfy the dataframe

In [ ]:
df_total_investments = pd.read_csv('../data/processed_data/total_investments_by_year.csv')
print(f'{min_year=}, {max_year=}')
for year in range(min_year, max_year+1):
    calculated_total = df_countries_investments[df_countries_investments['Year']==year]['Total Investments'].sum()
    real_total_invested_value = df_total_investments[df_total_investments['Year']==year]['USD'].sum()
    print(f'{year=}, Calculated total investment: {calculated_total}, Real total value: {real_total_invested_value}, assertion: {real_total_invested_value==real_total_invested_value}')
    # break

In [ ]:
# df_countries_investments[df_countries_investments['Year']==1998]['Total Investments'].sum()
# df_total_investments[df_total_investments['Year']==1998]['USD'].sum()